### Loading the trained CNN

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# Load your data
data = np.load('dataChestXray.npy')
target = np.load('targetChestXray.npy')

# Ensure target is in one-hot encoding format
if len(target.shape) == 1:
    target = to_categorical(target, num_classes=2)

# Define the model
def create_model(input_shape):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and train the model
input_shape = data.shape[1:]  # e.g., (128, 128, 1)
model = create_model(input_shape)

# Train the model
model.fit(data, target, epochs=10, batch_size=32, validation_split=0.2)

# Save the model
model_path = r'D:\Users\Amarjeet kumar\path\to\your\model\Pneumonia_predictions_using_chest_xray_99.20.h5'
model.save(model_path)


Epoch 1/10
131/131 [==============================] - 23s 168ms/step - loss: 0.5642 - accuracy: 0.7435 - val_loss: 0.4125 - val_accuracy: 0.8180
Epoch 2/10
131/131 [==============================] - 26s 200ms/step - loss: 0.3185 - accuracy: 0.8495 - val_loss: 0.2392 - val_accuracy: 0.9262
Epoch 3/10
131/131 [==============================] - 25s 194ms/step - loss: 0.2331 - accuracy: 0.9137 - val_loss: 0.2344 - val_accuracy: 0.9492
Epoch 4/10
131/131 [==============================] - 27s 204ms/step - loss: 0.1972 - accuracy: 0.9302 - val_loss: 0.1927 - val_accuracy: 0.9521
Epoch 5/10
131/131 [==============================] - 24s 186ms/step - loss: 0.1872 - accuracy: 0.9271 - val_loss: 0.1254 - val_accuracy: 0.9559
Epoch 6/10
131/131 [==============================] - 25s 191ms/step - loss: 0.1769 - accuracy: 0.9346 - val_loss: 0.1302 - val_accuracy: 0.9540
Epoch 7/10
131/131 [==============================] - 26s 196ms/step - loss: 0.1611 - accuracy: 0.9456 - val_loss: 0.1389 - val_ac

### Preprocessing the images to test&testing the model

In [ ]:
import cv2 as cv
import os
import numpy as np
import tensorflow as tf  # Ensure TensorFlow is installed

# Define the model loading function
def loadingCNN():
    model_path = r'D:\Users\Amarjeet kumar\path\to\your\model\Pneumonia_predictions_using_chest_xray_99.20.h5'
    if not os.path.isfile(model_path):
        print(f"File does not exist: {model_path}")
        return None

    try:
        model = tf.keras.models.load_model(model_path)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print(f"Failed to load model: {e}")
        return None

# Load the model
model = loadingCNN()
if model is None:
    raise RuntimeError("Model failed to load. Exiting.")

category = {0: 'Normal', 1: 'Pneumonia'}
testPath = 'Testimages'

valid_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.tiff'}

# Define the image processing function
def process_image(imagePath):
    image = cv.imread(imagePath)
    if image is None:
        print(f"Failed to load image: {imagePath}")
        return

    print(f"Processing image: {imagePath}")

    try:
        grayImage = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        resized = cv.resize(grayImage, (128, 128))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 128, 128, 1))
        
        result = model.predict(reshaped)
        label = np.argmax(result, axis=1)[0]
        prob = np.max(result, axis=1)[0]
        prob = round(prob, 2) * 100
        
        cv.putText(image, str(category[label]), (100, 100), cv.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
        cv.putText(image, f"{prob}%", (100, 200), cv.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
        cv.imshow('LIVE', image)
        cv.waitKey(5000)
        print(result)
    except Exception as e:
        print(f"An error occurred while processing the image {imagePath}: {e}")

# Define the directory processing function
def process_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            filePath = os.path.join(root, file)
            if any(filePath.lower().endswith(ext) for ext in valid_extensions):
                process_image(filePath)
            else:
                print(f"Skipping non-image file: {filePath}")

# Process images in the directory
process_directory(testPath)

cv.destroyAllWindows()

Model loaded successfully.
Processing image: Testimages\NORMAL\IM-0001-0001.jpeg
1/1 [==============================] - 0s 303ms/step
[[0.8025502  0.19744977]]
Processing image: Testimages\NORMAL\IM-0003-0001.jpeg
1/1 [==============================] - 0s 39ms/step
[[0.2082265  0.79177356]]
Processing image: Testimages\NORMAL\IM-0005-0001.jpeg
1/1 [==============================] - 0s 66ms/step
[[0.01914503 0.9808549 ]]
Processing image: Testimages\NORMAL\IM-0006-0001.jpeg
1/1 [==============================] - 0s 49ms/step
[[0.43716088 0.5628391 ]]
Processing image: Testimages\NORMAL\IM-0007-0001.jpeg
1/1 [==============================] - 0s 21ms/step
[[0.7465978 0.2534022]]
Processing image: Testimages\NORMAL\IM-0009-0001.jpeg
1/1 [==============================] - 0s 19ms/step
[[0.718087 0.281913]]
Processing image: Testimages\NORMAL\IM-0010-0001.jpeg
1/1 [==============================] - 0s 19ms/step
[[0.88753575 0.11246429]]
Processing image: Testimages\NORMAL\IM-0011-0001-0001.

[[0.0273442  0.97265583]]
Processing image: Testimages\NORMAL\IM-0102-0001.jpeg
1/1 [==============================] - 0s 38ms/step
[[0.13310781 0.8668922 ]]
Processing image: Testimages\NORMAL\IM-0103-0001.jpeg
1/1 [==============================] - 0s 39ms/step
[[0.9810028  0.01899718]]
Processing image: Testimages\NORMAL\IM-0105-0001.jpeg
1/1 [==============================] - 0s 38ms/step
[[0.2758984 0.7241016]]
Processing image: Testimages\NORMAL\IM-0107-0001.jpeg
1/1 [==============================] - 0s 38ms/step
[[0.65128106 0.348719  ]]
Processing image: Testimages\NORMAL\IM-0109-0001.jpeg
1/1 [==============================] - 0s 43ms/step
[[0.04774232 0.95225763]]
Processing image: Testimages\NORMAL\IM-0110-0001.jpeg
1/1 [==============================] - 0s 120ms/step
[[0.3172093 0.6827907]]
Processing image: Testimages\NORMAL\IM-0111-0001.jpeg
1/1 [==============================] - 0s 35ms/step
[[0.41938064 0.58061934]]
Processing image: Testimages\NORMAL\NORMAL2-IM-0007-0